# Compute mean, volatility, correlation of stock returns

In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from itertools import combinations
from tqdm import tqdm

from src.stock_data_manager import StockDataManager
from src.stock_statistics import StockStatistics
from src.utils import Utils

In [2]:
start_date = "2020-01-01"
end_date = "2024-10-01"
SP500_list_path = "data/tickers.json"

In [3]:
tickers = Utils.load_SP500_list(path=SP500_list_path)
tickers.remove("AMTM")  # abnormal data

In [4]:
sdm = StockDataManager(
    db_user=os.environ["MYSQL_USER"],
    db_password=os.environ["MYSQL_PASSWORD"],
    db_host="localhost",
    db_port="3306",
    db_name="STOCK_PROCESSED",
    alpaca_api_key=os.environ["ALPACA_API_KEY"],
    alpaca_secret_key=os.environ["ALPACA_SECRET_KEY"],
)

In [5]:
returns = {
    ticker: StockStatistics.calculate_mean_return(
        stock_data=sdm.load_from_db(ticker=ticker)
    )
    for ticker in tqdm(tickers)
}

  0%|          | 0/502 [00:00<?, ?it/s]

100%|██████████| 502/502 [00:15<00:00, 32.01it/s]


In [6]:
volatilities = {
    ticker: StockStatistics.calculate_volatility_using_GARCH(
        stock_data=sdm.load_from_db(ticker=ticker)
    )
    for ticker in tqdm(tickers)
}

100%|██████████| 502/502 [00:20<00:00, 24.74it/s]


In [ ]:
correlations = {
    Utils.make_ordered_pair(ticker1, ticker2): \
        StockStatistics.calculate_correlation(
        stock_data1=sdm.load_from_db(ticker=ticker1),
        stock_data2=sdm.load_from_db(ticker=ticker2),
    )
    for ticker1, ticker2 in tqdm(combinations(tickers, r=2))
}

125751it [1:58:37, 17.67it/s]
